In [1]:
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)
if 'NVIDIA' not in result.stdout:
    print('No GPU available. Exiting competition.')
    import sys
    sys.exit(1)

Fri Sep 26 00:33:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     128MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import subprocess
import sys
import os
import shutil
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Hard reset any prior torch stacks (and possible duplicates under PIP_TARGET)
for pkg in ('torch', 'torchvision', 'torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST (matches your CUDA 12.1 container)
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Create a constraints file to freeze torch versions for all later installs
Path('constraints.txt').write_text(
    'torch==2.4.1\n'
    'torchvision==0.19.1\n'
    'torchaudio==2.4.1\n'
)

# 3) Now install NON-torch deps, honoring constraints, and avoid upgrading torch
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity gate (hard fail on drift)
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version, 'cuda', '')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

# Additional packages for QA task
pip('install', 'indic-nlp-library', 'pyarrow', 'pandas', 'numpy')

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 513.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 221.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 264.7 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 KB 493.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 470.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 KB 473.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 522.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 KB 445.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 428.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 428.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 428.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 KB 474.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 KB 466.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 KB 462.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 KB 481.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 512.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 505.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 487.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 435.3 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
from collections import Counter

# Load training data
train_df = pd.read_csv('train.csv')
print('Train shape:', train_df.shape)
print('\nTrain columns:', train_df.columns.tolist())
print('\nTrain head:')
print(train_df.head())

# Check languages
print('\nLanguages distribution:')
print(train_df['language'].value_counts())

# Analyze answer_text
print('\nAnswer text lengths:')
train_df['answer_len'] = train_df['answer_text'].str.len()
print(train_df['answer_len'].describe())
print('\nUnique answers count:', train_df['answer_text'].nunique())

# Context and question lengths
train_df['context_len'] = train_df['context'].str.len()
train_df['question_len'] = train_df['question'].str.len()
print('\nContext lengths:')
print(train_df['context_len'].describe())
print('\nQuestion lengths:')
print(train_df['question_len'].describe())

# Sample rows
print('\nSample Hindi row:')
hindi_sample = train_df[train_df['language'] == 'hindi'].iloc[0]
print(f'Question: {hindi_sample["question"][:100]}...')
print(f'Context: {hindi_sample["context"][:100]}...')
print(f'Answer: {hindi_sample["answer_text"]}')

print('\nSample Tamil row:')
tamil_sample = train_df[train_df['language'] == 'tamil'].iloc[0]
print(f'Question: {tamil_sample["question"][:100]}...')
print(f'Context: {tamil_sample["context"][:100]}...')
print(f'Answer: {tamil_sample["answer_text"]}')

Train shape: (1002, 6)

Train columns: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language']

Train head:
          id                                            context  \
0  6bb0c472d  சிங்கம் என்பது பாலூட்டி வகையைச் சேர்ந்த ஒரு கா...   
1  34846a420  சென்னை (Chennai) தமிழ்நாட்டின் தலைநகரமும் இந்த...   
2  9d1c4fac8  एशियन पेंट्स लिमिटेड एक भारतीय बहुराष्ट्रीय कम...   
3  2b41f3744  भारत के पश्चिमी तट पर स्थित मुंंबई (पूर्व नाम ...   
4  7c73a8caa  एशिया या जम्बुद्वीप आकार और जनसंख्या दोनों ही ...   

                                            question  \
0          பெண் சிங்கம் என்று என்ன அழைக்கப்படுகிறது?   
1                சென்னை நகரம் எப்போது நிறுவப்பட்டது?   
2            एशियन पेंट्स कंपनी के संस्थापक कौन हैं?   
3  भारत का हिन्दी चलचित्र एवं दूरदर्शन उद्योग, कि...   
4                   एशिया का कुल क्षेत्रफल कितना है?   

                                         answer_text  answer_start language  
0                                            சிம்மம்     

# Initial Plan for Chaii Hindi-Tamil QA Competition

## Dataset Insights
- Train: 1002 samples (Hindi: 662, Tamil: 340) - imbalanced languages
- Standard extractive QA: predict answer_text and answer_start from context
- Contexts very long (mean 11k chars, max 49k) → need truncation or chunking for models (BERT max ~512 tokens)
- Answers short (mean 13 chars), 893 unique
- Metric: Word-level Jaccard score (intersection over union of word sets)

## Medal Targets
- Gold: ≥0.76305
- Silver: ≥0.73984
- Bronze: ≥0.73725
- Above median: ≥0.72756

## Strategy Outline
1. **Baseline Model**: Use provided starter notebook with multilingual BERT/XLM-R. Train separate models for Hindi/Tamil to handle imbalance.
2. **Data Preprocessing**: Tokenize with Indic tokenizer if needed; handle Devanagari/Tamil scripts via indic-nlp-library.
3. **Handle Long Contexts**: Truncate to 512 tokens or use sliding window/longformer variants.
4. **Cross-Validation**: 5-fold stratified by language; compute Jaccard on OOF predictions.
5. **Feature Engineering**: Language-specific features? Answer position stats, but focus on text embeddings.
6. **Modeling**: Fine-tune XLM-RoBERTa-large for QA; ensemble multilingual + language-specific (e.g., IndicBERT).
7. **Post-processing**: Normalize answers (e.g., remove punctuation for Jaccard).
8. **Iteration**: Start with subsample (e.g., 200 samples) for quick baselines; scale up.
9. **Ensembling**: Blend predictions from multiple models/seeds.
10. **Error Analysis**: Analyze OOF errors by language, answer length, context position.

## Next Steps
- Implement baseline model in new notebook (01_baseline.ipynb)
- Request expert review on medal-winning approaches, common pitfalls (e.g., tokenization issues with Indian scripts, CV leakage), and improvements over baseline.

In [4]:
# Fix fsspec incompatibility as per expert advice
import subprocess
import sys
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

pip('install', '-c', 'constraints.txt', 'fsspec[http]<=2024.6.1,>=2023.1.0', '--upgrade')

# Verify
import fsspec
print('fsspec version:', fsspec.__version__)
from datasets import load_dataset
print('Datasets import successful')

> install -c constraints.txt fsspec[http]<=2024.6.1,>=2023.1.0 --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 KB 7.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 129.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 KB 77.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.7/246.7 KB 485.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.3/235.3 KB 474.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 KB 438.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 KB 469.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 366.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 397.1 MB/s eta 0:00:00


fsspec version: 2024.6.1


Datasets import successful


/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
